In [1]:
import sys
import os

# Ruta al proyecto raíz donde está la carpeta 'scripts'
project_root = os.path.abspath('../')  # o usa la ruta absoluta si prefieres
sys.path.append(project_root)

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetimes
from dotenv import load_dotenv
from sqlalchemy import create_engine
from scripts.remove_outliers import remove_outliers

from sqlalchemy.orm import sessionmaker
pd.set_option('display.float_format', '{:.2f}'.format)


In [ ]:
query = """ 
BEGIN
    DECLARE @TotalRows INT;
    DECLARE @SampleSize INT;
    DECLARE @Q1 FLOAT, @Q3 FLOAT, @IQR FLOAT;
    DECLARE @LowerBound FLOAT, @UpperBound FLOAT;

    -- Contar el total de filas
    SELECT @TotalRows = COUNT(*)
    FROM dbo.BI_GAME_SESSIONS gs
    INNER JOIN dbo.BI_CARDS crd ON crd.CARD_ID = gs.CARD_ID
    INNER JOIN dbo.BI_PLAYERS pl ON pl.PLAYER_ID = crd.PLAYER_ID
    INNER JOIN dbo.BI_PLAYERS_LEVEL_ASSIGNED pll ON pll.PLAYER_ID = pl.PLAYER_ID
    WHERE gs.INITIAL_TIME >= '2024-01-01';

    -- Calcular tamaño de muestra
    SET @SampleSize = CEILING(@TotalRows * 0.1);

    -- Calcular Q1 y Q3 usando PERCENTILE_CONT
    WITH OrderedValues AS (
        SELECT CAST(gs.INITIAL_AMOUNT AS FLOAT) AS INITIAL_AMOUNT
        FROM dbo.BI_GAME_SESSIONS gs
        INNER JOIN dbo.BI_CARDS crd ON crd.CARD_ID = gs.CARD_ID
        INNER JOIN dbo.BI_PLAYERS pl ON pl.PLAYER_ID = crd.PLAYER_ID
        INNER JOIN dbo.BI_PLAYERS_LEVEL_ASSIGNED pll ON pll.PLAYER_ID = pl.PLAYER_ID
        WHERE gs.INITIAL_TIME >= '2024-01-01'
    )
    SELECT 
        @Q1 = PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY INITIAL_AMOUNT) OVER(),
        @Q3 = PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY INITIAL_AMOUNT) OVER()
    FROM OrderedValues;

    -- Calcular límites de outliers
    SET @IQR = @Q3 - @Q1;
    SET @LowerBound = @Q1 - 1.5 * @IQR;
    SET @UpperBound = @Q3 + 1.5 * @IQR;

    -- Seleccionar muestra filtrando outliers
    SELECT TOP (@SampleSize)
           pl.PLAYER_ID,
           pl.DOB,
           pl.GENDER,
           gs.AVG_BET,
           gs.BET_TOTAL,
           gs.INITIAL_AMOUNT,
           gs.INITIAL_TIME,
           gs.FINAL_TIME,
           gs.INITIAL_PROMO_AMOUNT,
           gs.FINAL_AMOUNT,
           gs.FINAL_PROMO_AMOUNT,
           gs.MACHINE_ID,
           gs.WIN_TOTAL,
           gs.GAMES_PLAYED_TOTAL,
           gs.GAMES_WON_TOTAL,
           gs.TIME_ON_DEVICE_SEC,
           pll.PLAYER_LEVEL_ID
    FROM dbo.BI_GAME_SESSIONS gs
    INNER JOIN dbo.BI_CARDS crd ON crd.CARD_ID = gs.CARD_ID
    INNER JOIN dbo.BI_PLAYERS pl ON pl.PLAYER_ID = crd.PLAYER_ID
    INNER JOIN dbo.BI_PLAYERS_LEVEL_ASSIGNED pll ON pll.PLAYER_ID = pl.PLAYER_ID
    WHERE gs.INITIAL_TIME >= '2024-01-01'
      AND gs.INITIAL_AMOUNT BETWEEN @LowerBound AND @UpperBound
    ORDER BY CHECKSUM(NEWID());
END

"""

In [4]:
load_dotenv()


server = os.getenv('SERVER')
username = os.getenv('SQL_USERNAME')
password = os.getenv('SQL_PASSWORD')

## Sendero ##
database = 'ewise_BI_Snd'
connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(connection_string)
Session = sessionmaker(bind=engine)
session = Session()
df_snd = pd.read_sql_query(query, engine)

## Irapuato ##
database = 'ewise_BI_Ira'
connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(connection_string)
Session = sessionmaker(bind=engine)
session = Session()
df_ira = pd.read_sql_query(query, engine)


## Satelite ##
database = 'ewise_BI_Stl'
connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(connection_string)
Session = sessionmaker(bind=engine)
session = Session()
df_stl = pd.read_sql_query(query, engine)

## Neza ##
database = 'ewise_BI_Nez'
connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(connection_string)
Session = sessionmaker(bind=engine)
session = Session()
df_nez = pd.read_sql_query(query, engine)

## Esmeralda ##
database = 'ewise_BI_Esm'
connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(connection_string)
Session = sessionmaker(bind=engine)
session = Session()
df_esm = pd.read_sql_query(query, engine)

## Allende ##
database = 'ewise_BI_All'
connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(connection_string)
Session = sessionmaker(bind=engine)
session = Session()
df_all = pd.read_sql_query(query, engine)

## Ajijic ##
database = 'ewise_BI_Ajj'
connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(connection_string)
Session = sessionmaker(bind=engine)
session = Session()
df_ajj = pd.read_sql_query(query, engine)

## Guadalupe ##
database = 'ewise_BI_Gpe'
connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(connection_string)
Session = sessionmaker(bind=engine)
session = Session()
df_gpe = pd.read_sql_query(query, engine)

## Mitras ##
database = 'ewise_BI_Mty'
connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(connection_string)
Session = sessionmaker(bind=engine)
session = Session()
df_mty = pd.read_sql_query(query, engine)




# General

In [5]:
df_snd['Casino'] = 'Sendero' 
df_ira['Casino'] = 'Irapuato'
df_stl['Casino'] = 'Satelite'
df_nez['Casino'] = 'Neza'
df_esm['Casino'] = 'Esmeralda'
df_all['Casino'] = 'Allende'
df_ajj['Casino'] = 'Ajjic'
df_gpe['Casino'] = 'Guadalupe'
df_mty['Casino'] = 'Mitras'


df = pd.concat([df_snd, df_ira, df_stl, df_nez, df_esm, df_all, df_ajj, df_gpe, df_mty])

In [6]:
df = df.loc[df['GAMES_PLAYED_TOTAL'] < 90000]
df.describe()

,AVG_BET,BET_TOTAL,INITIAL_AMOUNT,INITIAL_TIME,FINAL_TIME,INITIAL_PROMO_AMOUNT,FINAL_AMOUNT,FINAL_PROMO_AMOUNT,WIN_TOTAL,GAMES_PLAYED_TOTAL,GAMES_WON_TOTAL,TIME_ON_DEVICE_SEC,PLAYER_LEVEL_ID
count,281160.00,281149.00,281160.00,281160,281160,281160.00,281160.00,281160.00,281149.00,281160.00,281160.00,281153.00,281160.00
mean,4.67,811.46,270.33,2024-12-23 06:39:15.261431808,2024-12-23 06:54:04.817661440,2.32,224.14,1.74,142.93,194.67,58.80,889.45,4.05
min,0.00,0.00,0.00,2024-01-01 00:00:22.097000,2024-01-01 00:03:30.343000,0.00,0.00,0.00,-4999589.85,0.00,0.00,2.00,1.00
25%,2.11,116.75,109.33,2024-08-04 14:07:08.813999872,2024-08-04 14:25:55.869250048,0.00,1.04,0.00,38.50,36.00,9.00,169.00,2.00
50%,3.40,320.18,197.25,2024-12-27 23:39:21.059999744,2024-12-27 23:58:06.232000,0.00,9.25,0.00,177.00,89.00,25.00,408.00,4.00
75%,5.00,792.50,393.91,2025-05-05 21:47:21.809750016,2025-05-05 22:02:39.991500032,0.00,233.00,0.00,641.00,211.00,63.00,984.00,6.00
max,66647.81,277909.00,1732.31,2025-10-16 06:59:40.740000,2025-10-16 07:01:59.537000,6485.00,177953.30,3000.00,297368.75,10827.00,3791.00,82840.00,9.00
std,125.79,2167.51,210.26,NaN,NaN,30.51,1120.83,23.45,27334.22,334.98,106.48,1484.84,2.58


In [7]:
df = df.dropna()

In [8]:
df['DOB'] = pd.to_datetime(df['DOB'])
hoy = datetime.now()
df['Edad'] = hoy.year - df['DOB'].dt.year
df['Edad'] -= ((hoy.month < df['DOB'].dt.month) | 
               ((hoy.month == df['DOB'].dt.month) & 
                (hoy.day < df['DOB'].dt.day)))

bins = [18, 25, 35, 45, 55, 65, 100]  # 100 es un valor arbitrario para cubrir edades mayores a 65
labels = ['18-24', '25-34', '35-44', '45-54', '55-64', '65+']

# Crear la columna 'Rango_Edad'
df['Rango_Edad'] = pd.cut(df['Edad'], bins=bins, labels=labels, right=False)

In [11]:
from sklearn.preprocessing import LabelEncoder
le_edad = LabelEncoder()
df['Rango_Edad_le'] = le_edad.fit_transform(df['Rango_Edad'])

In [12]:
df.to_csv('../data_general_2.csv')